In [1]:
import numpy as np
import sklearn
import pandas as pd
import h5py

import matplotlib.pyplot as plt

from skimage.transform import resize, pyramid_gaussian
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

import zarr

import os
import sys
import re

import openslide

from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay



In [3]:
slide_meta = pd.read_csv("../metadata/labels_with_new_batch.csv")
ct_scoring = pd.read_csv("../metadata/CT_3_Class_Draft.csv")



ct_scoring["txt_idat"] = ct_scoring["idat"].astype("str")
ct_scoring.index = ct_scoring.txt_idat
slide_meta.index = slide_meta.idat
ct_scoring = ct_scoring.drop("txt_idat", axis=1)
slide_meta = slide_meta.drop("idat", axis=1)
slide_annots = slide_meta.join(ct_scoring, lsuffix="l")


myx = [x in ["Chromothripsis", "No Chromothripsis"] for x in slide_annots.CT_class]

slide_annots = slide_annots.loc[myx]
slide_names = slide_annots.uuid

# slide_names
slide_annots.CT_class


oncotree_map = pd.read_csv("../metadata/MappingClassifierToOncotree.csv")

slide_annots = slide_annots.merge(oncotree_map, left_on="max_super_family_class", right_on="Super Family", how="left")

slide_annots = slide_annots.loc[slide_annots['Oncotree code'].isin(['DIFG'])]
slide_annots = slide_annots.loc[slide_annots.max_cal_v11 * 100 >= 60]

slide_annots

,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,relevant Histo prediction,...,classifciation_v11,max_cal_v11,family,file_path,slide,Unnamed: 0,idat,CT_class,Super Family,Oncotree code
4,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,74022,"supratentoriell, frontal rechts",11.0,5459,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,"methylation class low grade glioma, dysembryop...",0.999950,PA and other MAPK LGGNT,NaN,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,76.0,10003886259_R02C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
8,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,73136,frontal left,24.0,5311,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant; high grade",...,"methylation class IDH glioma, subclass high gr...",0.965636,astrocytoma,NaN,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,494.0,10006823123_R01C01,Chromothripsis,Adult-type diffuse gliomas,DIFG
10,F488896F-A788-4EAA-9B34-DB063CD29189,73178,"supratentoriell, temporal links",8.0,5342,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,"methylation class low grade glioma, dysembryop...",0.999692,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,F488896F-A788-4EAA-9B34-DB063CD29189,510.0,10006823130_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
18,9DC7BE79-C914-44C4-B89A-5C8442D45C86,83010,infratentoriell,14.0,36924,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,infratentorial pilocytic astrocytoma,Infratentorial pilocytic astrocytoma,infratentorial pilocytic astrocytoma,...,"methylation class low grade glioma, subclass p...",0.998198,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,9DC7BE79-C914-44C4-B89A-5C8442D45C86,982.0,101178130116_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
21,6C3D45BF-80CC-495D-8B54-9095602228D7,83006,"supratentoriell, frontal rechts",15.0,36914,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,"methylation class low grade glioma, dysembryop...",0.979881,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,6C3D45BF-80CC-495D-8B54-9095602228D7,989.0,101178130116_R06C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,0A225B1C-4B3D-4534-95E8-D7B17CAF9894,70832,"supratentoriell, Nervus opticus",57.0,4277,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,supratentorial midline pilocytic astrocytoma,Supratentorial midline pilocytic astrocytoma,Supratentorial midline pilocytic astrocytoma,...,"methylation class low grade glioma, subclass m...",0.993623,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,0A225B1C-4B3D-4534-95E8-D7B17CAF9894,124321.0,9969477102_R05C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG
2176,9215A2D7-E714-476C-B04A-FC1C68E585AA,69456,Links temporal,3.0,3713,Low-grade glial/glioneuronal/neuroepithelial t...,"diffuse glioma, MYB- or MYBL1-altered","diffuse glioma, MYB(L1)-altered, subtype A [an...","Angiocentric glioma, MYB/MYBL1-altered","Angiocentric glioma, MYB/MYBL1-altered",...,"methylation class low grade glioma, MYB/MYBL1",0.999908,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,9215A2D7-E714-476C-B04A-FC1C6

In [4]:
slide_annots.idat.uniq()

AttributeError: 'Series' object has no attribute 'uniq'

In [5]:
slide_meta

,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,relevant Histo prediction,possible further consolidation,classifciation_v11,max_cal_v11,family,file_path,slide
idat,,,,,,,,,,,,,,,,
204920830120_R01C01,5E876B19-B3C5-42F0-9034-E171C9185A61,234138,"supratentoriell, temporo-frontal rechts",34.0,111977,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant","methylation class IDH glioma, subclass high gr...",0.912418,astrocytoma,/omics/odcf/analysis/OE0606_projects/pancancer...,5E876B19-B3C5-42F0-9034-E171C9185A61
204920830120_R01C01,FB077233-EDC3-4A52-BD6D-F2F330D7FA62,234138,"supratentoriell, temporo-frontal rechts",34.0,111977,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant","methylation class IDH glioma, subclass high gr...",0.912418,astrocytoma,/omics/odcf/analysis/OE0606_projects/pancancer...,FB077233-EDC3-4A52-BD6D-F2F330D7FA62
204920830120_R01C01,23A493E4-3A63-410B-9659-3AFDF2C366EE,234138,"supratentoriell, temporo-frontal rechts",34.0,111977,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant","methylation class IDH glioma, subclass high gr...",0.912418,astrocytoma,/omics/odcf/analysis/OE0606_projects/pancancer...,23A493E4-3A63-410B-9659-3AFDF2C366EE
207011010162_R07C01,7EC4A50F-B422-413F-983E-2418103F347F,326008,"supratentoriell, temporo-insulär rechts",37.0,156108,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; lower grade","Astrocytoma, IDH-mutant; lower grade","Astrocytoma, IDH-mutant","methylation class IDH glioma, subclass astrocy...",0.934602,astrocytoma,/omics/odcf/analysis/OE0606_projects/pancancer...,7EC4A50F-B422-413F-983E-2418103F347F
205566000169_R07C01,69DD0320-2930-49DD-9F3A-AAA43519D52A,258410,NaN,NaN,123570,"diffuse glioma, MAPK altered, cell-cycle activ...",pleomorphic xanthoastrocytoma(-like),pleomorphic xanthoastrocytoma(-like),Pleomorphic xanthoastrocytoma,Pleomorphic xanthoastrocytoma,Pleomorphic xanthoastrocytoma,methylation class (anaplastic) pleomorphic xan...,0.973061,pleomorphic xanthoastrocytoma,/omics/odcf/analysis/OE0606_projects/pancancer...,69DD0320-2930-49DD-9F3A-AAA43519D52A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204339010057_R06C01,872427CC-F757-4CEA-A33F-17B43C872535,173738,"supratentoriell, Thalamus links",44.0,83651,Paediatric-type diffuse high-grade gliomas,"Diffuse pediatric-type high-grade glioma, H3-w...","Diffuse paediatric-type high grade glioma, RTK...","Diffuse paediatric-type high grade glioma, RTK...","Diffuse paediatric-type high grade glioma, RTK...",Diffuse paediatric-type high grade glioma,methylation class CNS neuroblastoma with FOXR2...,0.092167,neuroblastoma,/omics/odcf/analysis/OE0606_projects/pancancer...,872427CC-F757-4CEA-A33F-17B43C872535
206947700046_R07C01,AAB165F9-6A45-4545-BCA3-67A0BE33AB26,317894,NaN,NaN,151989,Ependymal tumours,myxopapillary ependymoma,myxopapillary ependymoma,Myxopapillary ependymoma,Myxopapillary ependymoma,Myxopapillary ependymoma,NaN,NaN,NaN,/omics/odcf/analysis/OE0606_projects/pancancer...,AAB165F9-6A45-4545-BCA3-67A0BE33AB26
9741950087_R03C02,CADDF04F-C9A1-4A84-A840-1AA1053A982D,67754,"supratentoriell, frontal links",31.0,2105,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; lower grade","Astrocytoma, IDH-mutant; lower grade","Astrocytoma, IDH-mutant","methylation class IDH glioma, subclass astrocy...",0

In [6]:
path_to_extracted_features = '/home/p163v/histopathology/UKHD_Neuro/RetCLL_Features/'


patch_num = []
for x in slide_annots.uuid:
    fl = path_to_extracted_features+x+".h5"
    if(os.path.exists(fl)):
        patch_num.append(h5py.File(fl)['feats'].shape[0])
    else:
        patch_num.append(0)

In [7]:
slide_annots['patches'] = patch_num

In [8]:
slide_annots

,uuid,tumor_id,txt_LOKALISATION,num_ALTERSANGABE,patient_id,max_super_family_class,max_family_class,max_class,max_subclass,relevant Histo prediction,...,max_cal_v11,family,file_path,slide,Unnamed: 0,idat,CT_class,Super Family,Oncotree code,patches
4,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,74022,"supratentoriell, frontal rechts",11.0,5459,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,0.999950,PA and other MAPK LGGNT,NaN,88FB4DE0-39AE-4FD6-ACFA-68C35A57669F,76.0,10003886259_R02C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,75
8,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,73136,frontal left,24.0,5311,Adult-type diffuse gliomas,"diffuse glioma, IDH mutant","diffuse glioma, IDH-mutant and 1p19q retained ...","Astrocytoma, IDH-mutant; high grade","Astrocytoma, IDH-mutant; high grade",...,0.965636,astrocytoma,NaN,2064E961-D1F4-4EDC-B2AC-5801AD1D1ED3,494.0,10006823123_R01C01,Chromothripsis,Adult-type diffuse gliomas,DIFG,0
10,F488896F-A788-4EAA-9B34-DB063CD29189,73178,"supratentoriell, temporal links",8.0,5342,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,0.999692,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,F488896F-A788-4EAA-9B34-DB063CD29189,510.0,10006823130_R03C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,998
18,9DC7BE79-C914-44C4-B89A-5C8442D45C86,83010,infratentoriell,14.0,36924,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,infratentorial pilocytic astrocytoma,Infratentorial pilocytic astrocytoma,infratentorial pilocytic astrocytoma,...,0.998198,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,9DC7BE79-C914-44C4-B89A-5C8442D45C86,982.0,101178130116_R02C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,13197
21,6C3D45BF-80CC-495D-8B54-9095602228D7,83006,"supratentoriell, frontal rechts",15.0,36914,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade glioneuronal tumour,dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,Dysembryoplastic neuroepithelial tumour,...,0.979881,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,6C3D45BF-80CC-495D-8B54-9095602228D7,989.0,101178130116_R06C01,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,3548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,0A225B1C-4B3D-4534-95E8-D7B17CAF9894,70832,"supratentoriell, Nervus opticus",57.0,4277,Low-grade glial/glioneuronal/neuroepithelial t...,pilocytic astrocytoma,supratentorial midline pilocytic astrocytoma,Supratentorial midline pilocytic astrocytoma,Supratentorial midline pilocytic astrocytoma,...,0.993623,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,0A225B1C-4B3D-4534-95E8-D7B17CAF9894,124321.0,9969477102_R05C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,229
2176,9215A2D7-E714-476C-B04A-FC1C68E585AA,69456,Links temporal,3.0,3713,Low-grade glial/glioneuronal/neuroepithelial t...,"diffuse glioma, MYB- or MYBL1-altered","diffuse glioma, MYB(L1)-altered, subtype A [an...","Angiocentric glioma, MYB/MYBL1-altered","Angiocentric glioma, MYB/MYBL1-altered",...,0.999908,PA and other MAPK LGGNT,/omics/odcf/analysis/OE0606_projects/pancancer...,9215A2D7-E714-476C-B04A-FC1C68E585AA,124325.0,9969477104_R01C02,No Chromothripsis,Low-grade glial/glioneuronal/neuroepithelial t...,DIFG,2146
2179,B4C4055F-B3BC-4058-8697-7A2F3A0A2824,69462,Occipital,1.0,3715,Low-grade glial/glioneuronal/neuroepithelial t...,low-grade ganglioglial/neuroepithelial tumour,desmoplastic infantile ganglioglioma/astrocytoma,Desmoplastic infantile

In [7]:
# np.random.seed(42)

# slide_train, slide_valid = train_test_split(np.array(slide_annots.uuid), train_size = 0.6)

# slide_valid, slide_test = train_test_split(slide_valid, train_size = 0.5)

In [9]:
slide_annots['labels'] = np.abs(1-slide_annots.CT_class.factorize(sort=True)[0])

In [10]:
np.random.seed(42)


kfold = StratifiedKFold(5)
kfold_test_splits = [x for x in kfold.split(np.array(slide_annots.uuid), np.array(slide_annots.labels))]




In [11]:
kfold_train_valid_test_splits = [train_test_split(x[0], train_size=0.75, stratify = np.array(slide_annots.labels)[x[0]]) + [x[1]] for x in kfold_test_splits]




In [12]:
for i in range(5):
    print('starting: '+str(i))
    os.makedirs('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/02022024_DIFG/'+str(i),exist_ok=True)
    slide_train = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][0]]
    slide_train_files = path_to_extracted_features + slide_train + '.h5'
    slide_valid = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][1]]
    slide_valid_files = path_to_extracted_features + slide_valid + '.h5'
    slide_test = np.array(slide_annots.uuid)[kfold_train_valid_test_splits[i][2]]
    slide_test_files = path_to_extracted_features + slide_test + '.h5'
    
    patch_num_train = list()

    for fl in slide_train_files:
        if(os.path.exists(fl)):
            patch_num_train.append(h5py.File(fl)['feats'].shape[0])
        else:
            patch_num_train.append(0)
    
    train_df = pd.DataFrame({'slide': slide_train,
                             'features': slide_train_files,
                             'patches': patch_num_train})
    train_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/02022024_DIFG/'+str(i)+'/train_set.csv')
    
    patch_num_valid = list()

    for fl in slide_valid_files:
        if(os.path.exists(fl)):
            patch_num_valid.append(h5py.File(fl)['feats'].shape[0])
        else:
            patch_num_valid.append(0)
    
    valid_df = pd.DataFrame({'slide': slide_valid,
                             'features': slide_valid_files,
                             'patches': patch_num_valid})
    valid_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/02022024_DIFG/'+str(i)+'/valid_set.csv')
    
    patch_num_test = list()

    for fl in slide_test_files:
        if(os.path.exists(fl)):
            patch_num_test.append(h5py.File(fl)['feats'].shape[0])
        else:
            patch_num_test.append(0)
    
    test_df = pd.DataFrame({'slide': slide_test,
                             'features': slide_test_files,
                             'patches': patch_num_test})
    test_df.to_csv('/omics/odcf/analysis/OE0585_projects/chromothripsis/histopathology/splits/02022024_DIFG/'+str(i)+'/test_set.csv')
    

starting: 0
starting: 1
starting: 2
starting: 3
starting: 4


In [13]:
[x.shape for x in kfold_train_valid_test_splits[0]]

[(336,), (113,), (113,)]